GRUPO:

*   ALEXANDRE CICERO ARAUJO BEIRUTH
*   JOÃO VITOR GABARDO DA CUNHA
*   LEANDRO CARDOSO VIEIRA
*   LUANA TIEMANN HALICKI CORDEIRO

In [ ]:
import threading
import time

# Definindo a capacidade máxima da cozinha
MAX_PEDIDOS = 4

# Lista para armazenar os pedidos pendentes
pedidos_pendentes = []

# Contador para pedidos sequenciais
pedido_counter = 1

# Criação de um objeto condition para coordenar a execução das threads
condition = threading.Condition()

class Garcom(threading.Thread):
    def __init__(self, nome, total_pedidos=10):
        threading.Thread.__init__(self)
        self.nome = nome
        self.total_pedidos = total_pedidos

    def fazer_pedido(self):
        global pedido_counter
        with condition:
            while len(pedidos_pendentes) == MAX_PEDIDOS:
                # O garçom espera se a cozinha estiver cheia
                condition.wait()
            pedido = f'Pedido {pedido_counter}'
            pedidos_pendentes.append((self.nome, pedido))
            print(f'{self.nome} fez {pedido}')
            pedido_counter += 1
            # Notifica o cozinheiro que um pedido foi feito
            condition.notify_all()

    def run(self):
        for _ in range(self.total_pedidos):
            time.sleep(0.5)
            self.fazer_pedido()


class Cozinheiro(threading.Thread):
    def __init__(self, nome):
        threading.Thread.__init__(self)
        self.nome = nome

    def prepara_pedido(self):
        with condition:
            while len(pedidos_pendentes) == 0:
                # O cozinheiro espera se não houver pedidos pendentes
                condition.wait()
            garcom, pedido = pedidos_pendentes.pop(0)
            print(f'{self.nome} está preparando {pedido} do {garcom}')
            # Notifica os garçons que um pedido foi preparado
            condition.notify_all()

    def run(self):
        while True:
            time.sleep(0.5)
            if pedidos_pendentes:
                self.prepara_pedido()
            elif not any(garcom.is_alive() for garcom in garcons):
                break


def iniciar_restaurante(cozinheiros):
    global garcons, pedido_counter, pedidos_pendentes
    garcons = [Garcom(f'Garcom{i+1}') for i in range(5)]
    pedidos_pendentes = []
    pedido_counter = 1

    inicio = time.time()

    # Inicializando as threads
    for garcom in garcons:
        garcom.start()
    for cozinheiro in cozinheiros:
        cozinheiro.start()

    # Esperando todas as threads terminarem
    for garcom in garcons:
        garcom.join()
    for cozinheiro in cozinheiros:
        cozinheiro.join()

    fim = time.time()
    print("Todos os pedidos foram entregues e a cozinha está fechando.")
    print(f'Tempo total de execução com {len(cozinheiros)} cozinheiro(s): {fim - inicio} segundos')


# Primeiro, executamos com um cozinheiro
iniciar_restaurante([Cozinheiro('Cozinheiro1')])

# Em seguida, executamos com dois cozinheiros
iniciar_restaurante([Cozinheiro('Cozinheiro1'), Cozinheiro('Cozinheiro2')])


Garcom1 fez Pedido 1
Garcom2 fez Pedido 2
Garcom3 fez Pedido 3
Garcom4 fez Pedido 4
Cozinheiro1 está preparando Pedido 1 do Garcom1
Garcom5 fez Pedido 5
Cozinheiro1 está preparando Pedido 2 do Garcom2
Garcom1 fez Pedido 6
Cozinheiro1 está preparando Pedido 3 do Garcom3
Garcom2 fez Pedido 7
Cozinheiro1 está preparando Pedido 4 do Garcom4
Garcom3 fez Pedido 8
Cozinheiro1 está preparando Pedido 5 do Garcom5
Garcom3 fez Pedido 9
Cozinheiro1 está preparando Pedido 6 do Garcom1
Garcom4 fez Pedido 10
Cozinheiro1 está preparando Pedido 7 do Garcom2
Garcom1 fez Pedido 11
Cozinheiro1 está preparando Pedido 8 do Garcom3
Garcom1 fez Pedido 12
Cozinheiro1 está preparando Pedido 9 do Garcom3
Garcom2 fez Pedido 13
Cozinheiro1 está preparando Pedido 10 do Garcom4
Garcom2 fez Pedido 14
Cozinheiro1 está preparando Pedido 11 do Garcom1
Garcom3 fez Pedido 15
Cozinheiro1 está preparando Pedido 12 do Garcom1
Garcom1 fez Pedido 16
Cozinheiro1 está preparando Pedido 13 do Garcom2
Garcom1 fez Pedido 17
Cozinhe